In [1]:
import os
import numpy as np
import tensorflow as tf
import glob
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
wave_num=64
sz=256
def createGraph():
    rnd_f=tf.random_uniform([wave_num,1], 0.02, 0.1)
    rnd_p=np.pi*tf.random_uniform([wave_num,1])

    idx = tf.reshape(tf.constant(np.arange(sz,dtype=np.float32)),[1,sz])
    y=idx*rnd_f*np.pi+rnd_p
    return tf.sin(y) * 0.5

def outputWave(path):
    with tf.Session() as sess:
        g=createGraph()
        waves=sess.run(g)

        data = np.zeros([2,sz], dtype=np.float32)
        for i in range(len(waves)):
            fn = "{0}/{1:0>4}".format(path, i) + ".bin"
            data[0]=waves[i]
            data[1]=distortion(waves[i])
            data.tofile(fn)

def distortion(x):
    return np.sign(x)*(1-np.exp(-np.abs(x*8)))

In [3]:
with tf.Session() as sess:
    g=createGraph()
    waves=sess.run(g)

In [4]:
out_dir="./data/input/test"
os.makedirs(out_dir, exist_ok=True)
outputWave(out_dir)

In [5]:
input_paths = glob.glob(os.path.join(out_dir, "*.bin"))
input_paths.sort()

filename_queue = tf.train.string_input_producer(input_paths)
reader = tf.WholeFileReader()
key, value = reader.read(filename_queue)
image = tf.decode_raw(value, tf.float32)
image = tf.reshape(image,[2,1,tf.constant(sz),1])

sess=tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord)

out=sess.run(image)

coord.request_stop()
coord.join(threads)

xs = range(sz)
plt.plot(xs, out[0,0,:,0], xs, out[1,0,:,0])